In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

In [2]:
heros_full_features = pd.read_csv('dataset.csv')

# First: Separate the categorical from the boolean data


In [3]:
# Preprocessing
heros_nominal_information = heros_full_features.iloc[:, 2:11]
heros_boolean_information = heros_full_features.iloc[:, 11:]
heros_numerical_information = heros_full_features.loc[:, ['Height', 'Weight']]

In [4]:
# Remove Skin color, cause it is not good for the decision, then separete height and weight from the nominal atributes
heros_nominal_information = heros_nominal_information.drop(['Skin color', 'Height', 'Weight'], axis=1)

In [5]:
# Treat the missing numerical values giving then the mean of the others
imp = SimpleImputer(missing_values=-99, strategy='mean')
heros_numerical_information['Height'] = imp.fit_transform(np.array(heros_numerical_information['Height']).reshape(-1, 1))
heros_numerical_information['Weight'] = imp.fit_transform(np.array(heros_numerical_information['Weight']).reshape(-1, 1))

In [6]:
# Treat the missing nominal values giving then the most frequent of the others 
imp = SimpleImputer(missing_values='-', strategy='most_frequent')
# Labeling the nominal attributes 
le = preprocessing.LabelEncoder()
# All of this in the same loop

for index, item in heros_nominal_information.iteritems():
    heros_nominal_information.loc[:, index] = imp.fit_transform(np.array(heros_nominal_information.loc[:, index]).reshape(-1, 1))
    heros_nominal_information.loc[:, index] = le.fit_transform(heros_nominal_information.loc[:, index].astype(str))

In [7]:
# Discretize numerical attributes
#est = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
#heros_numerical_information = est.fit(heros_numerical_information)

In [8]:
pre_processed_dataset = pd.concat([heros_nominal_information, heros_boolean_information, heros_numerical_information], axis=1)

In [9]:
pre_processed_dataset.head()

,Gender,Eye color,Race,Hair color,Publisher,Alignment,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,...,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient,Height,Weight
0,1,18,22,14,11,1,False,True,False,False,...,False,False,False,False,False,False,False,False,203.000000,441.000000
1,1,2,31,14,2,1,True,True,False,False,...,False,False,False,False,False,False,False,False,191.000000,65.000000
2,1,2,53,14,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,185.000000,90.000000
3,1,7,26,14,11,0,False,True,False,False,...,False,False,False,False,False,False,False,False,203.000000,441.000000
4,1,2,10,1,11,0,False,False,False,True,...,False,False,False,False,False,False,False,False,187.237755,113.651709


In [10]:
# For Flight
target = pre_processed_dataset.loc[:, 'Flight']
x_train, x_test, y_train, y_test = train_test_split(pre_processed_dataset.drop(['Flight'], axis=1), target, test_size = 0.25, random_state = 42)

In [24]:
max_depths = np.linspace(1, 32, 32, endpoint=True)

train_results = []
test_results = []
for max_depth in max_depths:
   dt = DecisionTreeClassifier(max_depth=max_depth)
   dt.fit(x_train, y_train)
   train_pred = dt.predict(x_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous train results
   train_results.append(roc_auc)
   y_pred = dt.predict(x_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous test results
   test_results.append(roc_auc)
line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('Tree depth')
plt.show()

NameError: name 'precision_recall_curve' is not defined

In [23]:
tree_classifier = DecisionTreeClassifier(random_state = 0, max_depth = 26)
tree_classifier.fit(x_train, y_train)

# Filling all the NaN occurences, with the set mean
print('Accuracy on the train subset: ' + str(tree_classifier.score(x_train, y_train)))
print('Accuracy on the test subset: ' + str(tree_classifier.score(x_test, y_test)))

Accuracy on the train subset: 0.9858585858585859
Accuracy on the test subset: 0.7575757575757576


In [13]:
depth = []
for i in range(3,33):
    clf = DecisionTreeClassifier(max_depth=i, random_state = 0)
    # Perform 7-fold cross validation 
    scores = cross_val_score(estimator=clf, X=pre_processed_dataset.drop(['Flight'], axis=1), y=target, cv=7, n_jobs=4)
    depth.append((i,scores.mean()))
print(depth)

[(3, 0.771236602143657), (4, 0.7818908974564069), (5, 0.7742441209406495), (6, 0.7682290833466646), (7, 0.7742441209406495), (8, 0.766597344424892), (9, 0.7636218205087186), (10, 0.7484082546792513), (11, 0.7439449688049912), (12, 0.7424252119660855), (13, 0.7378659414493681), (14, 0.7227643577027676), (15, 0.7303151495760679), (16, 0.7242041273396257), (17, 0.7333546632538793), (18, 0.7136618141097425), (19, 0.7182050871860504), (20, 0.731818908974564), (21, 0.7136778115501521), (22, 0.7135658294672852), (23, 0.7105583106702928), (24, 0.7106223004319309), (25, 0.7121100623900175), (26, 0.7136298192289233), (27, 0.7105903055511117), (28, 0.7075507918733003), (29, 0.709070548712206), (30, 0.6984322508398656), (31, 0.7014717645176771), (32, 0.7029915213565829)]
